## 1. Import Libraries

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from ultralytics import YOLO
from tqdm import tqdm
import pandas as pd
import json
import shutil
from datetime import datetime
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances
import warnings
warnings.filterwarnings('ignore')

# Configure matplotlib for inline display in notebook
%matplotlib inline

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 10)

print('✓ Libraries imported successfully')

## 2. Configuration

In [ ]:
# Base directories
BASE_DIR = Path.cwd().parent

# from pathlib import Path
# BASE_DIR = Path("/computer_vision_yolo")

MODEL_NAME = "yolov8n"  # Model name without .pt extension

# Choose YOLO model versions that are fully supported with ultralytics:
# ✅ YOLOv8: 'yolov8n', 'yolov8s', 'yolov8m', 'yolov8l', 'yolov8x'
# ✅ YOLOv9: 'yolov9s', 'yolov9m', 'yolov9l', 'yolov9x'
# ✅ YOLOv10: 'yolov10n', 'yolov10s', 'yolov10m', 'yolov10l', 'yolov10x'
# ✅ YOLO11: 'yolo11n', 'yolo11s', 'yolo11m', 'yolo11l', 'yolo11x'
# ✅ YOLO12: 'yolo12n', 'yolo12s', 'yolo12m', 'yolo12l', 'yolo12x'

MODELS_DIR = BASE_DIR / 'models' / MODEL_NAME
TMP_DIR = BASE_DIR / 'tmp' / MODEL_NAME
RUNS_DIR = BASE_DIR / 'hyperparameter_tuning' / 'runs'

# Dataset Selection - Choose one:
# Option 1: Full dataset (~100k images) - use for final optimization
# YOLO_DATASET_ROOT = BASE_DIR / 'bdd100k_yolo'
# DATA_YAML_PATH = YOLO_DATASET_ROOT / 'data.yaml'

# Option 2: Limited dataset (representative samples - recommended for quick tuning)
YOLO_DATASET_ROOT = BASE_DIR / 'bdd100k_yolo_limited'
DATA_YAML_PATH = YOLO_DATASET_ROOT / 'data.yaml'

# Verify dataset exists
if not DATA_YAML_PATH.exists():
    raise FileNotFoundError(
        f"Dataset not found: {DATA_YAML_PATH}\n\n"
        f"Please run the dataset preparation script first:\n"
        f"  python3 process_bdd100k_to_yolo_dataset.py\n"
    )

# Tuning configuration
N_TRIALS = 20  # Number of optimization trials
TIMEOUT_HOURS = 4  # Maximum time for optimization

# Generate timestamp and run name
RUN_TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
RUN_NAME = f'{MODEL_NAME}_tuning_{RUN_TIMESTAMP}'
W_B_PROJECT = "yolo-bdd100k-hyperparameter-tuning"

# Create run-specific directory
RUN_DIR = RUNS_DIR / RUN_NAME
RUN_DIR.mkdir(parents=True, exist_ok=True)

# Create other directories
TMP_DIR.mkdir(parents=True, exist_ok=True)
RUNS_DIR.mkdir(parents=True, exist_ok=True)

# Read number of classes from data.yaml
import yaml
with open(DATA_YAML_PATH, 'r') as f:
    data_config = yaml.safe_load(f)
    NUM_CLASSES = data_config['nc']
    CLASS_NAMES = {i: name for i, name in enumerate(data_config['names'])}

print('✓ Configuration loaded')
print(f'  Model: {MODEL_NAME}')
print(f'  Dataset: {YOLO_DATASET_ROOT.name}')
print(f'  Data YAML: {DATA_YAML_PATH}')
print(f'  Run directory: {RUN_DIR}')
print(f'  Classes: {NUM_CLASSES}')
print(f'  Optimization trials: {N_TRIALS}')
print(f'  Timeout: {TIMEOUT_HOURS} hours')

## 3. Load Base YOLO Model

In [ ]:
# Load YOLO model with automatic download
model_path = MODELS_DIR / f'{MODEL_NAME}.pt'

if not model_path.exists():
    print(f'Model not found at {model_path}')
    print(f'Downloading {MODEL_NAME} ...')
    
    try:
        # Download model
        MODEL_NAME_n = MODEL_NAME
        if MODEL_NAME.startswith('yolo11') or MODEL_NAME.startswith('yolo12'):
            MODEL_NAME_n = MODEL_NAME + '.pt'
        model = YOLO(MODEL_NAME_n)
        
        # Create models directory
        MODELS_DIR.mkdir(parents=True, exist_ok=True)
        
        # Move from cache
        try:
            import glob
            cache_patterns = [
                str(Path.home() / '.cache' / 'ultralytics' / '**' / f'{MODEL_NAME}.pt'),
                str(Path.home() / '.config' / 'Ultralytics' / '**' / f'{MODEL_NAME}.pt'),
            ]
            
            model_found = False
            for pattern in cache_patterns:
                cache_paths = glob.glob(pattern, recursive=True)
                if cache_paths:
                    shutil.move(cache_paths[0], model_path)
                    print(f'✓ Model downloaded and saved to {model_path}')
                    print(f'  Size: {model_path.stat().st_size / (1024*1024):.1f} MB')
                    
                    # Clean up cache directory
                    cache_dir = Path(cache_paths[0]).parent
                    if cache_dir.exists() and not any(cache_dir.iterdir()):
                        cache_dir.rmdir()
                        print(f'  ✓ Cleaned up cache directory')
                    
                    model_found = True
                    break
            
            if not model_found:
                print(f'✓ Model loaded from ultralytics cache')
        except Exception as save_error:
            print(f'⚠️  Could not move model: {save_error}')
            print(f'✓ Model loaded successfully from cache')
            
    except Exception as e:
        print(f'\n❌ Error downloading model: {e}')
        raise
else:
    model = YOLO(str(model_path))
    print(f'✓ Model loaded from {model_path}')

print(f'\n📊 Base Model: {MODEL_NAME}')
print(f'  Classes in model: {len(model.names)}')
print(f'  Task: {model.task}')

## 4. Verify Dataset Structure

In [ ]:
# Verify dataset structure is ready
print('Verifying YOLO dataset structure...')
print(f'\n📁 Dataset Root: {YOLO_DATASET_ROOT}')

# Check splits
for split in ['train', 'val', 'test']:
    images_dir = YOLO_DATASET_ROOT / 'images' / split
    labels_dir = YOLO_DATASET_ROOT / 'labels' / split
    
    if images_dir.exists() and labels_dir.exists():
        num_images = len(list(images_dir.glob('*.jpg'))) + len(list(images_dir.glob('*.png')))
        num_labels = len(list(labels_dir.glob('*.txt')))
        print(f'  ✓ {split:5s}: {num_images:6d} images, {num_labels:6d} labels')
    else:
        print(f'  ⚠️  {split:5s}: Directory not found')

# Verify data.yaml
print(f'\n📄 Configuration: {DATA_YAML_PATH}')
print(f'  Classes: {NUM_CLASSES}')
print(f'  Names: {CLASS_NAMES}')

print('\n✓ Dataset structure verified and ready for training')

## 5. Define Hyperparameter Search Space

In [ ]:
def define_hyperparameters(trial):
    """
    Define the hyperparameter search space for YOLO optimization.
    Conservative ranges around YOLO defaults to ensure training stability.
    
    Returns:
        dict: Hyperparameters to test
    """
    
    # Training hyperparameters with conservative ranges
    hyperparams = {
        # Optimizer settings - narrower ranges around proven defaults
        'lr0': trial.suggest_float('lr0', 0.0005, 0.01, log=True),  # Initial learning rate (default ~0.01)
        'lrf': trial.suggest_float('lrf', 0.01, 0.2),  # Final learning rate factor (default 0.01)
        'momentum': trial.suggest_float('momentum', 0.85, 0.98),  # SGD momentum (default 0.937)
        'weight_decay': trial.suggest_float('weight_decay', 0.0001, 0.001, log=True),  # (default 0.0005)
        
        # Data augmentation - moderate values
        # 'hsv_h': trial.suggest_float('hsv_h', 0.0, 0.02),  # HSV Hue (default 0.015)
        # 'hsv_s': trial.suggest_float('hsv_s', 0.3, 0.9),  # HSV Saturation (default 0.7)
        # 'hsv_v': trial.suggest_float('hsv_v', 0.2, 0.6),  # HSV Value (default 0.4)
        # 'degrees': trial.suggest_float('degrees', 0.0, 10.0),  # Rotation (default 0.0)
        # 'translate': trial.suggest_float('translate', 0.05, 0.2),  # Translation (default 0.1)
        # 'scale': trial.suggest_float('scale', 0.0, 0.5),  # Scaling (default 0.5)
        # 'flipud': trial.suggest_float('flipud', 0.0, 0.1),  # Flip up-down (default 0.0)
        # 'fliplr': trial.suggest_float('fliplr', 0.3, 0.7),  # Flip left-right (default 0.5)
        # 'mosaic': trial.suggest_float('mosaic', 0.5, 1.0),  # Mosaic augmentation (default 1.0)
        # 'mixup': trial.suggest_float('mixup', 0.0, 0.1),  # Mixup augmentation (default 0.0)
        
        # Loss function weights - stay close to defaults
        'box': trial.suggest_float('box', 6.0, 8.5),  # Box loss weight (default 7.5)
        'cls': trial.suggest_float('cls', 0.4, 0.7),  # Classification loss weight (default 0.5)
        'dfl': trial.suggest_float('dfl', 1.3, 1.7),  # DFL loss weight (default 1.5)
        
        # Training settings
        'warmup_epochs': trial.suggest_int('warmup_epochs', 2, 5),  # (default 3)
        'warmup_momentum': trial.suggest_float('warmup_momentum', 0.7, 0.95),  # (default 0.8)
        'warmup_bias_lr': trial.suggest_float('warmup_bias_lr', 0.05, 0.15),  # (default 0.1)
        
        # Fixed parameters - increased for better learning
        'epochs': 50,  # Increased for better convergence
        'batch': 16,
        'imgsz': 640,
        'patience': 15,  # More patience for convergence
        'save': False,  # Don't save intermediate models
        'plots': False,  # Don't generate plots for each trial
        'cache': True,  # Cache images for faster training
        'workers': 8,  # Use multiple workers
        'close_mosaic': 10,  # Disable mosaic in last 10 epochs for better convergence
    }
    
    return hyperparams

print('✓ Hyperparameter search space defined')
print('  Strategy: Conservative ranges around YOLO defaults')
print('  Parameters to optimize:')
print('    - Learning rate (lr0, lrf) - narrow range')
print('    - Optimizer (momentum, weight_decay) - near defaults')
print('    - Data augmentation (hsv, geometric, mosaic, mixup) - moderate')
print('    - Loss weights (box, cls, dfl) - close to defaults')
print('    - Warmup settings')
print('  Fixed settings:')
print('    - Epochs: 50 (increased for better convergence)')
print('    - Batch: 16')
print('    - Image size: 640')
print('    - Patience: 15')

## 6. Define Objective Function

In [ ]:
def objective(trial):
    """
    Objective function for Optuna optimization.
    Trains a YOLO model with trial hyperparameters and returns validation mAP@0.5.
    
    Args:
        trial: Optuna trial object
        
    Returns:
        float: Validation mAP@0.5 (metric to maximize)
    """
    
    # Get hyperparameters for this trial
    hyperparams = define_hyperparameters(trial)
    
    # Create trial-specific directory
    trial_dir = RUN_DIR / f'trial_{trial.number:03d}'
    trial_dir.mkdir(exist_ok=True)
    
    try:
        # Initialize W&B for this trial
        try:
            import wandb
            wandb.init(
                project=W_B_PROJECT,
                name=f'{MODEL_NAME}_trial_{trial.number:03d}',
                config=hyperparams,
                reinit=True
            )
        except:
            wandb = None
        
        # Load fresh model for this trial
        model = YOLO(str(model_path))
        
        # Print trial info
        print(f'\n[Trial {trial.number}] Training with:')
        print(f'  lr0={hyperparams["lr0"]:.6f}, lrf={hyperparams["lrf"]:.4f}')
        print(f'  momentum={hyperparams["momentum"]:.4f}, weight_decay={hyperparams["weight_decay"]:.6f}')
        print(f'  box={hyperparams["box"]:.2f}, cls={hyperparams["cls"]:.2f}, dfl={hyperparams["dfl"]:.2f}')
        
        # Train with hyperparameters using data.yaml directly
        results = model.train(
            data=str(DATA_YAML_PATH),
            project=str(trial_dir),
            name='train',
            exist_ok=True,
            verbose=False,
            **hyperparams
        )
        
        # Get validation metrics
        val_results = model.val(
            data=str(data_yaml_path),
            split=USED_DATA_SPLIT,
            verbose=False
        )
        
        # Extract mAP@0.5 as optimization metric
        map50 = float(val_results.box.map50)
        map50_95 = float(val_results.box.map)
        precision = float(val_results.box.mp)
        recall = float(val_results.box.mr)
        
        # Log to W&B
        if wandb:
            wandb.log({
                'val/mAP@0.5': map50,
                'val/mAP@0.5:0.95': map50_95,
                'val/precision': precision,
                'val/recall': recall
            })
            wandb.finish()
        
        # Save trial results
        trial_results = {
            'trial_number': trial.number,
            'hyperparameters': hyperparams,
            'metrics': {
                'map50': map50,
                'map50_95': map50_95,
                'precision': precision,
                'recall': recall
            }
        }
        
        with open(trial_dir / 'results.json', 'w') as f:
            json.dump(trial_results, f, indent=2)
        
        print(f'Trial {trial.number}: mAP@0.5={map50:.4f}, mAP@0.5:0.95={map50_95:.4f}')
        
        return map50
        
    except Exception as e:
        print(f'  ❌ Trial {trial.number} failed with error: {e}')
        import traceback
        traceback.print_exc()
        if wandb:
            wandb.finish()
        return 0.0  # Return poor score for failed trials

print('✓ Objective function defined')
print('  Optimization metric: mAP@0.5 (to maximize)')
print('  Additional tracking: mAP@0.5:0.95, precision, recall')
print('  Validation: Uses training results + explicit validation if needed')
print('  Error handling: Detailed logging and fallback to small non-zero scores')

## 7. Run Hyperparameter Optimization

In [ ]:
print('=' * 80)
print('STARTING HYPERPARAMETER OPTIMIZATION')
print('=' * 80)
print(f'Model: {MODEL_NAME}')
print(f'Dataset: {USED_DATASET} - {USED_DATA_SPLIT} split')
print(f'Number of trials: {N_TRIALS}')
print(f'Timeout: {TIMEOUT_HOURS} hours')
print(f'Strategy: Conservative ranges + longer training (50 epochs)')
print('=' * 80)

# Create Optuna study with more lenient pruning
study = optuna.create_study(
    study_name=f'{MODEL_NAME}_tuning',
    direction='maximize',  # Maximize mAP@0.5
    sampler=optuna.samplers.TPESampler(seed=42, n_startup_trials=5),  # More random exploration
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=8,  # Don't prune first 8 trials
        n_warmup_steps=20,  # Wait 20 steps before pruning
        interval_steps=5  # Check every 5 steps
    )
)

# Run optimization
study.optimize(
    objective,
    n_trials=N_TRIALS,
    timeout=TIMEOUT_HOURS * 3600,
    show_progress_bar=True
)

print('\n' + '=' * 80)
print('OPTIMIZATION COMPLETED')
print('=' * 80)
print(f'Total trials: {len(study.trials)}')
print(f'Best trial: {study.best_trial.number}')
print(f'Best mAP@0.5: {study.best_value:.4f}')
print('=' * 80)

## 8. Analyze Best Hyperparameters

In [ ]:
# Get best hyperparameters
best_params = study.best_params

print('=' * 80)
print('BEST HYPERPARAMETERS')
print('=' * 80)
print(json.dumps(best_params, indent=2))
print('=' * 80)

# Save best parameters
best_params_path = RUN_DIR / 'best_hyperparameters.json'
with open(best_params_path, 'w') as f:
    json.dump({
        'best_trial': study.best_trial.number,
        'best_value': study.best_value,
        'hyperparameters': best_params,
        'timestamp': datetime.now().isoformat()
    }, f, indent=2)

print(f'\n✓ Best hyperparameters saved to: {best_params_path}')

## 9. Visualize Optimization Results

In [ ]:
# Plot optimization history
fig1 = plot_optimization_history(study)
fig1.update_layout(title=f'{MODEL_NAME} Hyperparameter Optimization History')
fig1.show()

# Save figure
optimization_history_path = RUN_DIR / 'optimization_history.png'
fig1.write_image(str(optimization_history_path))
print(f'✓ Optimization history saved to: {optimization_history_path}')

In [ ]:
# Plot parameter importances
try:
    fig2 = plot_param_importances(study)
    fig2.update_layout(title=f'{MODEL_NAME} Hyperparameter Importance')
    fig2.show()
    
    # Save figure
    param_importance_path = RUN_DIR / 'parameter_importance.png'
    fig2.write_image(str(param_importance_path))
    print(f'✓ Parameter importance saved to: {param_importance_path}')
except (RuntimeError, ValueError) as e:
    print(f'⚠️  Could not generate parameter importance plot: {e}')
    print('  This can happen when trials have insufficient data variation.')
    param_importance_path = None

## 10. Create Results Summary

In [ ]:
# Create comprehensive results summary
trials_data = []
for trial in study.trials:
    trials_data.append({
        'trial': trial.number,
        'mAP@0.5': trial.value,
        'state': trial.state.name,
        **trial.params
    })

df_trials = pd.DataFrame(trials_data)
df_trials = df_trials.sort_values('mAP@0.5', ascending=False)

print('=' * 80)
print('TOP 10 TRIALS')
print('=' * 80)
print(df_trials.head(10).to_string(index=False))
print('=' * 80)

# Save trials summary
trials_csv_path = RUN_DIR / 'trials_summary.csv'
df_trials.to_csv(trials_csv_path, index=False)
print(f'\n✓ Trials summary saved to: {trials_csv_path}')

## 11. Save Hyperparameters for Training

In [ ]:
print('=' * 80)
print('SAVING BEST HYPERPARAMETERS FOR TRAINING')
print('=' * 80)

# Create training directory
training_dir = BASE_DIR / 'training'
training_dir.mkdir(exist_ok=True)

# Prepare training hyperparameters with recommended settings
training_params = best_params.copy()
training_params.update({
    'epochs': 100,  # Full training epochs
    'batch': 16,
    'imgsz': 640,
    'patience': 20,
    'save': True,
    'plots': True,
    'verbose': True,
    'cache': True,
    'workers': 8,
    'project': 'training',  # Project directory
    'name': MODEL_NAME,  # Experiment name
    'exist_ok': True
})

# Save hyperparameters for training
training_params_path = training_dir / f'{MODEL_NAME}_best_hyperparameters.json'
with open(training_params_path, 'w') as f:
    json.dump({
        'model': MODEL_NAME,
        'base_model_path': str(model_path),
        'dataset_root': str(YOLO_DATASET_ROOT),
        'data_yaml_path': str(DATA_YAML_PATH),
        'optimization_results': {
            'best_trial': study.best_trial.number,
            'best_map50': study.best_value,
            'total_trials': len(study.trials)
        },
        'hyperparameters': training_params,
        'timestamp': datetime.now().isoformat(),
        'notes': 'Use these hyperparameters for full model training'
    }, f, indent=2)

print(f'✓ Training hyperparameters saved to: {training_params_path}')
print(f'\nRecommended training command:')
print(f'  model = YOLO("{model_path}")')
print(f'  model.train(data="{DATA_YAML_PATH}", **hyperparameters)')
print('\nHyperparameters optimized for:')
for key, value in best_params.items():
    print(f'  {key}: {value}')
print('=' * 80)

## 12. Generate PDF Report

In [ ]:
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors as rl_colors
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_CENTER, TA_LEFT
from PIL import Image as PILImage

print('=' * 80)
print('GENERATING PDF REPORT')
print('=' * 80)

# Create PDF report
pdf_report_path = RUN_DIR / f'{MODEL_NAME}_hyperparameter_tuning_report.pdf'

doc = SimpleDocTemplate(str(pdf_report_path), pagesize=A4,
                       rightMargin=30, leftMargin=30,
                       topMargin=30, bottomMargin=30)

story = []
styles = getSampleStyleSheet()

# Custom styles
title_style = ParagraphStyle(
    'CustomTitle',
    parent=styles['Heading1'],
    fontSize=24,
    textColor=rl_colors.HexColor('#2c3e50'),
    spaceAfter=30,
    alignment=TA_CENTER
)

heading_style = ParagraphStyle(
    'CustomHeading',
    parent=styles['Heading2'],
    fontSize=16,
    textColor=rl_colors.HexColor('#34495e'),
    spaceAfter=12,
    spaceBefore=20
)

# Title
story.append(Paragraph(f'{MODEL_NAME} Hyperparameter Tuning Report', title_style))
story.append(Spacer(1, 12))

# Configuration info
info_data = [
    ['Model:', MODEL_NAME],
    ['Dataset:', f'{USED_DATASET} - {USED_DATA_SPLIT} split'],
    ['Timestamp:', datetime.now().strftime('%Y-%m-%d %H:%M:%S')],
    ['Total Trials:', str(len(study.trials))],
    ['Best Trial:', str(study.best_trial.number)],
    ['Best mAP@0.5:', f'{study.best_value:.4f}']
]

info_table = Table(info_data, colWidths=[2.2*inch, 3.8*inch])
info_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, -1), rl_colors.HexColor('#ecf0f1')),
    ('TEXTCOLOR', (0, 0), (-1, -1), rl_colors.black),
    ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
    ('FONTNAME', (0, 0), (0, -1), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, -1), 10),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 8),
    ('TOPPADDING', (0, 0), (-1, -1), 8),
    ('GRID', (0, 0), (-1, -1), 1, rl_colors.white)
]))
story.append(info_table)
story.append(Spacer(1, 20))

# Best hyperparameters
story.append(PageBreak())
story.append(Paragraph('Best Hyperparameters', heading_style))

hyperparam_data = [['Parameter', 'Value']]
for key, value in best_params.items():
    hyperparam_data.append([key, f'{value:.6f}' if isinstance(value, float) else str(value)])

hyperparam_table = Table(hyperparam_data, colWidths=[3*inch, 3*inch])
hyperparam_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), rl_colors.HexColor('#3498db')),
    ('TEXTCOLOR', (0, 0), (-1, 0), rl_colors.whitesmoke),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 12),
    ('FONTSIZE', (0, 1), (-1, -1), 9),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
    ('TOPPADDING', (0, 0), (-1, -1), 6),
    ('ROWBACKGROUNDS', (0, 1), (-1, -1), [rl_colors.white, rl_colors.lightgrey]),
    ('GRID', (0, 0), (-1, -1), 1, rl_colors.black)
]))
story.append(hyperparam_table)
story.append(Spacer(1, 20))

# Top 10 trials
story.append(PageBreak())
story.append(Paragraph('Top 10 Trials', heading_style))

top10_data = [['Trial', 'mAP@0.5', 'State']]
for _, row in df_trials.head(10).iterrows():
    top10_data.append([
        str(int(row['trial'])),
        f"{row['mAP@0.5']:.4f}",
        row['state']
    ])

top10_table = Table(top10_data, colWidths=[1.5*inch, 2*inch, 2.5*inch])
top10_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), rl_colors.HexColor('#27ae60')),
    ('TEXTCOLOR', (0, 0), (-1, 0), rl_colors.whitesmoke),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 11),
    ('FONTSIZE', (0, 1), (-1, -1), 9),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
    ('TOPPADDING', (0, 0), (-1, -1), 6),
    ('ROWBACKGROUNDS', (0, 1), (-1, -1), [rl_colors.white, rl_colors.lightgrey]),
    ('GRID', (0, 0), (-1, -1), 1, rl_colors.black)
]))
story.append(top10_table)
story.append(Spacer(1, 20))

# Optimization history
story.append(PageBreak())
story.append(Paragraph('Optimization History', heading_style))
story.append(Spacer(1, 12))

if optimization_history_path.exists():
    try:
        with PILImage.open(optimization_history_path) as img:
            img_width, img_height = img.size
            aspect_ratio = img_height / img_width
            pdf_width = 6.5 * inch
            pdf_height = pdf_width * aspect_ratio
            if pdf_height > 7 * inch:
                pdf_height = 7 * inch
                pdf_width = pdf_height / aspect_ratio
            story.append(Image(str(optimization_history_path), width=pdf_width, height=pdf_height))
    except Exception as e:
        print(f'Warning: Could not load optimization history: {e}')
        story.append(Paragraph('Optimization history chart not available.', styles['Normal']))
else:
    story.append(Paragraph('Optimization history chart not found.', styles['Normal']))

story.append(Spacer(1, 20))

# Parameter importance
story.append(PageBreak())
story.append(Paragraph('Parameter Importance', heading_style))
story.append(Spacer(1, 12))

if param_importance_path and param_importance_path.exists():
    try:
        with PILImage.open(param_importance_path) as img:
            img_width, img_height = img.size
            aspect_ratio = img_height / img_width
            pdf_width = 6.5 * inch
            pdf_height = pdf_width * aspect_ratio
            if pdf_height > 7 * inch:
                pdf_height = 7 * inch
                pdf_width = pdf_height / aspect_ratio
            story.append(Image(str(param_importance_path), width=pdf_width, height=pdf_height))
    except Exception as e:
        print(f'Warning: Could not load parameter importance: {e}')
        story.append(Paragraph('Parameter importance chart not available.', styles['Normal']))
else:
    story.append(Paragraph('Parameter importance chart not available or could not be generated.', styles['Normal']))

story.append(Spacer(1, 20))

# Footer
story.append(Spacer(1, 30))
story.append(Paragraph('Generated by YOLO Hyperparameter Tuning Notebook',
                      ParagraphStyle('Footer', parent=styles['Normal'],
                                   alignment=TA_CENTER, textColor=rl_colors.grey)))
story.append(Paragraph('BDD100K Dataset - Computer Vision Project',
                      ParagraphStyle('Footer2', parent=styles['Normal'],
                                   alignment=TA_CENTER, textColor=rl_colors.grey)))

# Build PDF
doc.build(story)

print(f'✓ PDF report generated: {pdf_report_path}')
print(f'  Size: {pdf_report_path.stat().st_size / 1024:.2f} KB')
print('=' * 80)

## 13. Summary

In [ ]:
print('=' * 80)
print('HYPERPARAMETER TUNING COMPLETED')
print('=' * 80)
print(f'\nModel: {MODEL_NAME}')
print(f'Dataset: {USED_DATASET} - {USED_DATA_SPLIT} split')
print(f'\nOptimization Results:')
print(f'  Total trials: {len(study.trials)}')
print(f'  Best trial: {study.best_trial.number}')
print(f'  Best mAP@0.5: {study.best_value:.4f}')
print(f'\nOutput Directory: {RUN_DIR}')
print(f'\nGenerated Files:')
print(f'  📄 Best hyperparameters: best_hyperparameters.json')
print(f'  📊 Trials summary: trials_summary.csv')
print(f'  📈 Optimization history: optimization_history.png')
print(f'  📊 Parameter importance: parameter_importance.png')
print(f'  📄 PDF Report: {MODEL_NAME}_hyperparameter_tuning_report.pdf')
print(f'  🎯 Training config: ../training/{MODEL_NAME}_best_hyperparameters.json')
print(f'\n💾 All results saved to: {RUN_DIR}')
print(f'📊 W&B Project: {W_B_PROJECT}')
print(f'\n🚀 Next Steps:')
print(f'  1. Review the PDF report')
print(f'  2. Use hyperparameters from training/{MODEL_NAME}_best_hyperparameters.json')
print(f'  3. Train your model with the optimized hyperparameters')
print('=' * 80)